# Question 15

In [1]:
from scipy.linalg import expm
import numpy as np
def renewal_pdf(x, lam):
    return lam**2 * x * np.exp(-lam * x)

def renewal_cdf(x, lam):
    return np.exp(-lam * x) * (1 + lam * x)
lam = 4.0
xs = np.linspace(0, 100, 1000)
ps = renewal_pdf(xs, lam) 
ps = ps / np.sum(ps)

def sample_renewal(n):
    return np.random.choice(xs, p=ps, size = n)

def phase_type_pdf(alpha, S, s ,  t):
    mp1 = len(alpha)
    return alpha[:(mp1-1)] @ expm(S * t) @ s

def phase_type_cdf(alpha, S, t):
    mp1 = len(alpha)
    one = np.ones(mp1-1)
    return 1- alpha[:(mp1-1)] @ (expm(S * t) @ one)

def add_phase_type(alpha, S, s, beta, T, t):
    m = len(alpha)-1
    k = len(beta)-1
    print(m, k)
    L = np.zeros((m + k, m + k))
    l = np.zeros(m + k)
    L[:m, :m] = S
    L[m:, :m] = np.outer(s, beta[:k])
    L[m:, m:] = T
    L[:m, m:] = np.zeros((k, m))
    l[:m] = s * beta[-1]
    l[m:] = t
    alpha = np.catenatecon([alpha[:m], alpha[-1] * beta])
    return alpha, L, l

alpha = np.array([1/4, 3/4, 0])
lambda_s = 1/3
lambda_u = 4
S = np.array([[-lambda_s, 0], [0, -lambda_u]])
s = np.array([lambda_s, lambda_u])

ts = np.linspace(0, 100, 10000)
ps_phase = np.zeros(len(ts))
for i, time in enumerate(ts):
    ps_phase[i] = phase_type_pdf(alpha, S, s, time)
ps_phase = ps_phase / np.sum(ps_phase)

def sample_phase_type(ts = ts, ps = ps_phase, n=1):
    return np.random.choice(ts, p=ps, size = n)


def check_phase_type(k_max, max_time, n = 500):
    succeses = np.zeros(k_max+1)

    for i in range(n):
        for k in range(0, k_max+1):
            time = 0
            admissions = 0
            extra_care_times = sample_renewal(k)
            for phase_change in extra_care_times:
                time += phase_change
                if time > max_time:
                    break
                # Sample time to absorption in PH
                time_to_absorption = sample_phase_type()
                time += time_to_absorption
                if time > max_time:
                    break
                admissions += 1
            if admissions == k:
                # Sample once more to ensure that we actually have k admissions and not k+1
                extra_care_time = sample_renewal(1)
                time_to_absorption = sample_phase_type()
                if extra_care_time + time_to_absorption + time > max_time:
                    succeses[k] += 1
    return succeses

In [2]:
succeses = check_phase_type(24, 36)

pss = succeses / 500
p_25_or_more = 1 - np.sum(pss)
print("Probability of 25 or more admissions in 3 years")
print(p_25_or_more)

Probability of 25 or more admissions in 3 years
0.5640000000000001


# Qustion 16

In [3]:
lengths = np.array([1/2, 3 * 1/4, 1/4 * 3/4])
p = lengths / np.sum(lengths)
p2 = lengths[1:] / np.sum(lengths[1:])
S = np.array([[-lambda_s, 0], [0, -lambda_u]])
s = np.array([lambda_s, lambda_u])
alpha_new = np.array([p2[0], p2[1] , 0])

P = phase_type_cdf(alpha_new, S, 2) #P(X <= 2)
P2 = check_phase_type(0, 2) 
print(P, 1-P2[0] / 500)
print((1-p[0]) * P + (1-P2[0])/500 * p[0])
print(p2)

0.5891992122483458 0.806
0.3174777471184864
[0.8 0.2]
